In [46]:
import pandas as pd
import numpy as np
import tabula
pd.options.display.max_rows = None

In [147]:
def lecturas_gomez(file_name):

    conv = 2.833 #Ajuste entre valor en mm obtenido por Acrobat y valores aceptados por Tabula

    #Valores en mm
    area_1=56.6
    area_2=13
    area_3=259
    area_4=206

    column_1=41.3
    column_2=55.25
    column_3=70
    column_4=85
    column_5=98
    column_6=129.5
    column_7=149.4
    column_8=160.55
    column_9=173.15
    column_10=190.41
    column_11=203

    df = tabula.read_pdf(file_name,
                         pages='all',
                         guess=True,
                         multiple_tables=False,
                         area=(area_1*conv,area_2*conv,area_3*conv,area_4*conv),
                         columns=(column_1*conv,
                                  column_2*conv,
                                  column_3*conv,
                                  column_4*conv,
                                  column_5*conv,
                                  column_6*conv,
                                  column_7*conv,
                                  column_8*conv,
                                  column_9*conv,
                                  column_10*conv,
                                  column_11*conv),
                         pandas_options={'dtype': str, 'header':None, 'skiprows':5})

    df.rename(columns={0:'Concepto',
                       1:'Fecha_anterior',
                       2:'Lectura_anterior',
                       3:'Lectura_actual',
                       4:'Consumo',
                       5:'Incidencias',
                       6:'Otros',
                       7:'Cuota_fija',
                       8:'Cuota_lect',
                       9:'Cuota_consumo',
                       10:'Total_recibo'
                       }, inplace=True)

    #Limpieza de filas no utilizables
    df = df[(df.Concepto!='LOCALIZACION') 
            & (df.Concepto!='Dirección de Lectura')
            & (df.Concepto!='Fecha')
            & (df.Concepto.notnull())
            & (df.Concepto!='Anterior')].reset_index(drop=True)

    
    #Eliminamos las últimas filas y asignamos la vivienda en función de si incluye la lectura de 1 o 2 conceptos.
    #Al ser 100 viviendas  en este punto en el primer caso tendremos en torno a 200 filas y en el segundo en torno a 300
    if len(df['Concepto'])>250: #Caso 2 conceptos
        df = df.drop(df.index[-3:])
        
        #Creamos la columna 'Vivienda', y le asignamos el valor de la vivienda correspondiente.La formula depende del valor en el
        #campo 'Concepto'
        df['Vivienda'] = np.where(df['Concepto']=='AGUA CALIENTE', df['Concepto'].shift(-2), df['Concepto'].shift(-1))
    
    else: #Caso 1 concepto
        df = df.drop(df.index[-2:])

        #Creamos la columna 'Vivienda', y le asignamos el valor de la vivienda correspondiente.
        df['Vivienda'] = df['Concepto'].shift(-1)
    
    #Eliminamos las filas sin concepto utilizando el campo vacio de fecha
    df = df[df['Fecha_anterior'].notnull()].reset_index(drop=True)
    
    #Colocamos la columna 'Vivienda' al principio del dataframe
    a = list(df.columns)
    df = df[a[-1:] + a[:-1]]
       
    #Eliminamos el símbolo de € en los campos monetarios y transformamos a float
    df['Total_recibo'] = df['Total_recibo'].str.replace(" €", "")
    df['Total_recibo'] = df['Total_recibo'].str.replace(",", ".").astype(float)
    df['Cuota_consumo'] = df['Cuota_consumo'].str.replace(" €", "").astype(float)
    df['Cuota_lect'] = df['Cuota_lect'].str.replace(" €", "").astype(float)
    df['Cuota_fija'] = df['Cuota_fija'].str.replace(" €", "").astype(float)
    df['Consumo'] = df['Consumo'].str.replace(" M3", "")
    df['Consumo'] = df['Consumo'].str.replace(" KWH", "").astype(int)
    
    #Obtenemos e introducimos la fecha de lectura
    
    area_date_1=14
    area_date_2=182
    area_date_3=19
    area_date_4=200

    df_date = tabula.read_pdf(file_name,
                     pages=1,
                     guess=True,
                     multiple_tables=False,
                     area=(area_date_1*conv,area_date_2*conv,area_date_3*conv,area_date_4*conv),
                     pandas_options={'dtype': str, 'header':None})

    fecha_lectura=df_date.iloc[0,0]
    df['Fecha_lectura'] = fecha_lectura
    
    #Colocamos la columna 'Fecha_lectura' en tercera posición
    a = list(df.columns)
    df = df[a[:3] + a[-1:] + a[3:-1]]
    
    #Formato campos temporales
    df['Fecha_lectura']=pd.to_datetime(df['Fecha_lectura'],format="%d/%m/%Y")
    df['Fecha_anterior']=pd.to_datetime(df['Fecha_anterior'],format="%d/%m/%y")
    
    return(df)

In [148]:
from glob import glob
lecturas=pd.DataFrame()
for counter, file in enumerate(glob('*.pdf')):
    df=lecturas_gomez(file)
    lecturas=lecturas.append(df)
    print('Leído %d de %d' % (counter+1,len(glob('*.pdf'))))
lecturas.to_excel('./lecturas.xls', index=False)
print('Guardado sin errores como "lecturas.xls"')

Leído 1 de 11
Leído 2 de 11
Leído 3 de 11
Leído 4 de 11
Leído 5 de 11
Leído 6 de 11
Leído 7 de 11
Leído 8 de 11
Leído 9 de 11
Leído 10 de 11
Leído 11 de 11
Guardado sin errores como "lecturas.xls"
